***

### Logic
#### Backlog giao: Bưu cục nhận hàng giao từ tải => Giao thành công/Trả hàng
    * Không quá 120h.
#### Backlog trả: Bưu cục nhận hàng trả từ tải => Trả hàng thành công
    * Không quá 72h.
#### Backlog lấy
    * Không quá 72h?
#### Backlog luân chuyển lấy: Lấy hàng thành công => Bàn giao tải 
    * HCM -> HN: 12h.
    * Các tỉnh còn lại: 32h.
#### Backlog luân chuyển trả: Chuyển trạng thái trả hàng => Bàn giao tải
    * Giao hàng tại HCM HN: 36h.
    * Giao hàng tại các tỉnh còn lại: 56h.
***

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import pandas as pd
import numpy as np
from datetime import datetime
import os


# change current working directory to Downloads
os.chdir('/Users/phuoc/Downloads')

# read data
export = pd.read_excel('export.xlsx')
inside = pd.read_excel('inside.xlsx')

# old_name: new_name -> replace inside headers
renamed_inside_headers = {
    'Mã đơn': 'MaDH',
    'Mã kiện': 'MaKien',
    'Kho gửi': 'KhoGui',
    'Kho nhận': 'KhoNhan',
    'Kho hiện tại': 'KhoHienTai',
    'TG đóng kiện': 'TGDongKien',
    'TG cập nhật': 'TGCapNhat',
    'TG nhận kiện': 'TGNhanKien',
    'TG kết thúc': 'TGKetThuc',
    'Trạng thái': 'TrangThaiLuanChuyen',
    'Số đơn': 'SoDon',
    'Khối lượng': 'KhoiLuong',
    'Mã niêm phong đóng': 'MaNiemPhongDong',
    'Mã niêm phong nhận': 'MaNiemPhongNhan',
    'Hình thức đóng gói': 'HinhThucDongGoi',
    'Hình thức vận chuyển': 'HinhThucVanChuyen',
    'Ghi chú': 'GhiChu'
}

# rename columns name of inside dataframe
inside = inside.rename(columns=renamed_inside_headers)

# normalize datetime of inside dataframe
# inside['TGDongKien'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y %H:%M:%S'))

# change str to datetime
inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]] = inside[[
    'TGDongKien',
    'TGCapNhat',
    'TGNhanKien',
    'TGKetThuc'
]].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y %H:%M:%S'))
inside['KhoGui'] = inside['KhoGui'].astype(int)
inside['KhoNhan'] = inside['KhoNhan'].astype(int)
inside['KhoHienTai'] = inside['KhoHienTai'].astype(int)

# normalize datetime of export dataframe
# replace all <nil> values
export = export.replace('<nil>', np.nan)

# change str to datetime
export[[
    'ThoiGianTao',
    'ThoiGianTaoChuyenDoi',
    'ThoiGianKetThucLay',
    'ThoiGianGiaoLanDau',
    'ThoiGianKetThucGiao',
    'ThoiGianGiaoHangMongMuon',
    'TGKetThucTra',
]] = export[[
        'ThoiGianTao',
        'ThoiGianTaoChuyenDoi',
        'ThoiGianKetThucLay',
        'ThoiGianGiaoLanDau',
        'ThoiGianKetThucGiao',
        'ThoiGianGiaoHangMongMuon',
        'TGKetThucTra',
]].apply(pd.to_datetime)

# set display of dataframe
pd.set_option('display.max_columns', 88)
pd.set_option('display.max_rows', 100)

# join export with inside
data = pd.merge(export, inside[['MaDH','MaKien', 'KhoGui', 'KhoNhan', 'TGNhanKien', 'TrangThaiLuanChuyen']], on='MaDH', how='left')

# codes of e-commerces exchange
shopee_codes = ['18692']
sendo_codes = ['1539', '1160902', '1160904', '1160905']
tiki_codes = ['1367']
lazada_codes = ['1041351', '9794']
ecoms = shopee_codes + sendo_codes + tiki_codes + lazada_codes

# classify types of backlog
# delivery backlog (backlog giao)
delivery_filter = data['TrangThai'].isin([
    'Đang giao hàng', 'Giao hàng không thành công'
]) | (((data['TrangThai'] == 'Lưu kho') | (data['TrangThai'] == 'Lấy hàng thành công') | (data['TrangThai'] == 'Đang trung chuyển hàng')) &\
      (data['KhoGiao'] == data['KhoHienTai'])) | ((data['TrangThai'] == 'Chờ giao lại') & (datetime.now() - data['ThoiGianKetThucGiao'] > pd.Timedelta('24 hours')))

delivery = data.loc[delivery_filter]
delivery['LoaiBacklog'] = 'Kho giao'
internal_delivery_filter = delivery['KhoLay'] == delivery['KhoHienTai']
delivery.loc[internal_delivery_filter, 'N0'] = delivery.loc[internal_delivery_filter]['ThoiGianKetThucLay']
delivery.loc[~internal_delivery_filter, 'N0'] = delivery.loc[~internal_delivery_filter]['TGNhanKien']
delivery['N+'] = delivery['N0'] + pd.Timedelta(hours=120)
delivery['Aging'] = (datetime.today() - delivery['N+']).fillna(pd.Timedelta(hours=9999))


# return backlog (backlog trả)
returned_filter = data['TrangThai'].isin([
    'Đang hoàn hàng', 'Hoàn hàng không thành công'
    ]) | (((data['TrangThai'] == 'Chuyển hoàn') | (data['TrangThai'] == 'Đang trung chuyển hàng hoàn')) & (((~data['KhoTra'].isnull()) & (data['KhoTra'] == data['KhoHienTai'])) |\
                                                 ((data['KhoTra'].isnull()) & (data['KhoLay'] == data['KhoHienTai']))))
returned = data.loc[returned_filter]
returned['LoaiBacklog'] = 'Kho trả'
internal_return_filter = returned['KhoHienTai'] == returned['KhoGiao']
returned.loc[internal_return_filter, 'N0'] = returned.loc[internal_return_filter]['ThoiGianKetThucGiao']
returned.loc[~internal_return_filter, 'N0'] = returned.loc[~internal_return_filter]['TGNhanKien']
returned['N+'] = returned['N0'] + pd.Timedelta(hours=72)
returned['Aging'] = (datetime.today() - returned['N+']).fillna(pd.Timedelta(hours=9999))


# pickup backlog (backlog lấy)
pickup_filter = (data['TrangThai'].isin(['Chờ lấy hàng', 'Đang lấy hàng', 'Lấy không thành công'])) |\
              ((data['TrangThai'] == 'Tạo thành công') & (data['ThoiGianKetThucLay'].isnull()) &\
               (data['KhoHienTai'] == data['KhoLay']))
pickup = data.loc[pickup_filter]
pickup['LoaiBacklog'] = 'Kho lấy'
ecoms_filter = pickup['MaKH'].isin(ecoms)
pickup.loc[ecoms_filter, 'N0'] = pickup.loc[ecoms_filter]['ThoiGianTaoChuyenDoi']
pickup.loc[~ecoms_filter, 'N0'] = pickup.loc[~ecoms_filter]['ThoiGianTao']
pickup['N+'] = pickup['N0'] + pd.Timedelta(hours=72)
pickup['Aging'] = (datetime.today() - pickup['N+']).fillna(pd.Timedelta(hours=9999))

# transporting backlog (backlog luân chuyển)
trans_filter = (((data['TrangThai'] == 'Lấy hàng thành công') | (data['TrangThai'] == 'Lưu kho')) & (data['KhoGiao'] != data['KhoHienTai'])) |\
                       ((data['TrangThai'] == 'Đang trung chuyển hàng') & (data['KhoLay'] != data['KhoGiao']))
transporting = data.loc[trans_filter]
transporting['LoaiBacklog'] = 'Kho lấy luân chuyển'
transporting['N0'] = transporting['ThoiGianKetThucLay']
transporting['N+'] = transporting['N0']+ pd.Timedelta(hours=12) # Kho lay tai HCM or HN
# transporting['N+'] = transporting['N0'] + pd.Timedelta(hours=32) # Kho lay tai cac tinh con lai
transporting['Aging'] = (datetime.today() - transporting['N+']).fillna(pd.Timedelta(hours=9999))


# returned_transporting backlog (backlog luân chuyển trả hàng)
rt_filter = ((data['TrangThai'].isin(['Chuyển hoàn', 'Chờ chuyển hoàn']) | (data['TrangThai'] == 'Đang trung chuyển hàng hoàn')) &\
             ((data['KhoTra'].isnull() & (data['KhoLay'] != data['KhoHienTai'])) |
              (~data['KhoTra'].isnull() & (data['KhoTra'] != data['KhoHienTai']))))
r_transporting = data.loc[rt_filter]
r_transporting['LoaiBacklog'] = 'Kho giao luân chuyển'
r_transporting['N0'] = r_transporting['ThoiGianKetThucGiao']
r_transporting['N+'] = r_transporting['N0'] + pd.Timedelta(hours=36) # Kho giao tai HCM or HN
# r_transporting['N+'] = r_transporting['N0'] + pd.Timedelta(hours=56) # Kho giao tai cac tinh con lai
r_transporting['Aging'] = (datetime.today() - r_transporting['N+']).fillna(pd.Timedelta(hours=9999))


# composite data
data = delivery.append([returned, pickup, transporting, r_transporting])
# set e-commerces column to NaN
data['ECommerces'] = 'Others'


# name of e-commerces exchange
# shopee
data.loc[data['MaKH'].isin(shopee_codes), 'ECommerces'] = 'Shopee'
# sendo
data.loc[data['MaKH'].isin(sendo_codes), 'ECommerces'] = 'Sendo'
# tiki
data.loc[data['MaKH'].isin(tiki_codes), 'ECommerces'] = 'Tiki'
# lazada
data.loc[data['MaKH'].isin(lazada_codes), 'ECommerces'] = 'Lazada'

# ToanTrinh
data['Aging_ToanTrinh'] = np.nan
ecoms_filter = data['MaKH'].isin(ecoms)
data.loc[ecoms_filter, 'Aging_ToanTrinh'] = datetime.today() - data[ecoms_filter]['ThoiGianTaoChuyenDoi']
data.loc[~ecoms_filter, 'Aging_ToanTrinh'] = datetime.today() - data[~ecoms_filter]['ThoiGianTao']
# reset the index of datafram
data.reset_index(inplace=True)
del data['index']
data['KhoGui'].astype('int64', errors='ignore')
data['NgayHienTai'] = datetime.today().strftime('%Y-%m-%d')
data.drop_duplicates(inplace=True)
data
# data['KhoGui']

,MaDH,MaDHShop,ThoiGianTao,ThoiGianTaoChuyenDoi,ThoiGianKetThucLay,ThoiGianGiaoLanDau,ThoiGianKetThucGiao,ThoiGianGiaoHangMongMuon,TGKetThucTra,ThongTinTraHang,TuQuan,TuTinh,TuVung,DenQuan,DenTinh,DenVung,KhoLay,KhoGiao,KhoHienTai,KhoTra,TrangThai,TrongLuong,KichThuoc,MaKH,GhiChuGHN,GhiChu,GoiCuoc,SoLanLay,SoLanGiao,SoLanTra,TinhTrangLuanChuyen,MaNVLay,MaNVGiao,TenNguoiGui,MaDiaChiLay,MaKien,KhoGui,KhoNhan,TGNhanKien,TrangThaiLuanChuyen,LoaiBacklog,N0,N+,Aging,ECommerces,Aging_ToanTrinh,NgayHienTai
0,8FH5DH471Y,NaN,2020-08-22 13:44:58,2020-08-22 13:44:58,2020-08-23 15:36:03,NaT,NaT,2020-08-25 16:10:00,NaT,NaN,Quận 10,Hồ Chí Minh,E,Quận 7,Hồ Chí Minh,E,Bưu Cục 16A Lê Hồng Phong-Q.10-HCM 04,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 16A Lê Hồng Phong-Q.10-HCM 04,Lưu kho,500,10x10x10,218664,'22/08/2020 09:17:05 - GHN-PFA1A0|THAY ĐỔI THÔ...,'',1 Ngày,2,0,0,NaN,1704104,NaN,'Nhinhi Nguyen',235990,20ZCU37873,2329.0,2275.0,2020-08-23 23:34:34,Kết thúc,Kho giao,2020-08-23 23:34:34,2020-08-28 23:34:34,18 days 00:23:13.252110,Others,24 days 10:12:49.374686,2020-09-15
1,8FH64XX9N6,5f4bb392bb80950001e07025_18,2020-08-30 21:32:22,2020-08-31 08:00:00,2020-08-31 16:01:41,2020-09-07 10:10:07,NaT,2020-09-07 08:18:00,NaT,NaN,Huyện Đan Phượng,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Bưu Cục 8 Đình Nguyên-Đan Phượng-HN,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 8 Đình Nguyên-Đan Phượng-HN,Lưu kho,2000,15x10x10,813308,'07/09/2020 03:31:54 - GHN-DFC1A0|THAY ĐỔI THÔ...,"'HÀNG DỄ VỠ XIN NHẸ TAY, KHÁCH KHÔNG NHẬN HÀN...",2 Ngày,1,4,0,NaN,1874485,NaN,'Vân Trần',811421,20DME65968,2329.0,2275.0,2020-09-03 14:08:01,Kết thúc,Kho giao,2020-09-03 14:08:01,2020-09-08 14:08:01,7 days 09:49:46.252110,Others,16 days 02:25:25.374686,2020-09-15
2,8FHKQL4HF9,NaN,2020-08-28 16:45:41,2020-08-29 08:00:00,2020-08-28 18:28:14,2020-08-29 15:18:10,NaT,2020-09-01 17:54:00,NaT,NaN,Quận Hoàng Mai,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Kho Khách hàng lớn Hà Nội,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Kho Khách hàng lớn Hà Nội,Lưu kho,500,10x10x10,1090576,'29/08/2020 09:12:38 - GHN-DFC1A7|THAY ĐỔI THÔ...,'Nếu khách muốn xem hàng bưu tá gọi điện lại c...,2 Ngày,1,1,0,NaN,2031724,NaN,'Chất Shop',933794,20VBY64496,2329.0,2275.0,2020-08-29 15:01:49,Kết thúc,Kho giao,2020-08-29 15:01:49,2020-09-03 15:01:49,12 days 08:55:58.252110,Others,18 days 07:12:06.374686,2020-09-15
3,8FHLF1DDXH,S1903175O101,2020-08-27 23:42:08,2020-08-28 08:00:00,2020-08-28 19:49:07,NaT,NaT,2020-09-01 10:42:00,NaT,NaN,Quận Gò Vấp,Hồ Chí Minh,E,Quận 7,Hồ Chí Minh,E,Bưu Cục 1284 Quang Trung-Q.Gò Vấp-HCM,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 1284 Quang Trung-Q.Gò Vấp-HCM,Lưu kho,210,10x5x10,1278224,NaN,'',1 Ngày,1,0,0,NaN,2039684,NaN,'MỸ PHẨM _diệp',1290115,20DRP93012,2329.0,2275.0,2020-08-30 02:45:19,Kết thúc,Kho giao,2020-08-30 02:45:19,2020-09-04 02:45:19,11 days 21:12:28.252110,Others,19 days 00:15:39.374686,2020-09-15
4,8FHQSYUK56,2428158453018627385,2020-08-31 06:53:09,2020-08-31 08:00:00,2020-08-31 15:20:25,2020-09-04 12:47:13,2020-09-13 23:22:50,2020-09-05 01:06:00,NaT,'Hàng giao chậm quá thời gian cam kết',Huyện Thanh Trì,Hà Nội,G,Quận 7,Hồ Chí Minh,E,Bưu Cục Cầu Bươu-ThanhTrì-HN,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục 37 Bế Văn Cấm-Q.7-HCM 01,Bưu Cục Cầu Bươu-ThanhTrì-HN,Chờ giao lại,3500,10x10x10,18692,'04/09/2020 06:12:50 - GHN-DFC1A0|THAY ĐỔI THÔ...,'Không cho xem hàng. Khối lượng nhận hàng tối ...,2 Ngày,1,1,0,NaN,1600107,NaN,'congtrangbedding.com',1154025,20KEG07873,2268.0,2275.0,2020-09-03 20:53:00,Kết thúc,Kho giao,2020-09-03 20:53:00,2020-09-08 20:53:00,7 days 03:04:47.252110,Shopee,15 days 15:57:47.337922,2020-09-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5815,9FIHUY3K4,2434868827738112768,2020-09-09 13:05:27,2020-09-12 08:00:00,2020-09-12 16

In [10]:
# export data backlog to csv file
# data export
data_to_export = data[[
    'NgayHienTai',
    'MaDH',
    'KhoHienTai',
    'TrangThai',
    'GhiChuGHN',
    'SoLanLay',
    'SoLanGiao',
    'SoLanTra',
    'LoaiBacklog',
    'N0',
    'N+',
    'Aging',
    'Aging_ToanTrinh',
    'ECommerces',
    'TrangThaiLuanChuyen',
]]

data_to_export.to_csv('backlog.csv', encoding='utf-8-sig', index=False)


In [11]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as df2g


# define scope of google api json keyfile
SCOPES = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]

# credentials to google service account
credentials = ServiceAccountCredentials.from_json_keyfile_name('ggapi_key.json', scopes=SCOPES)
# authorize with gspread
gc = gspread.authorize(credentials)

SPREADSHEET_ID = '1_UlAs1Ed6rxl1TNM8Vv2fnEsRT61BOWTVzCMtd1hwME'  # id of sheets (on url)
WKS_NAME = 'Backlog_bot'
active_sheets = gc.open_by_key(SPREADSHEET_ID)
wks = active_sheets.worksheet(WKS_NAME)
data_upload = data_to_export.loc[data_to_export['Aging'] >= pd.Timedelta(hours=0)]
# update data to google sheets
wks.clear()
df2g.upload(
    data_upload,
    SPREADSHEET_ID,
    WKS_NAME,
    credentials=credentials,
    row_names=False,
)


<Worksheet 'Backlog_bot' id:181055436>

In [12]:
# read dataframe from google sheet
from df2gspread import gspread2df as g2df

df_gspread = g2df.download(gfile=SPREADSHEET_ID, wks_name=WKS_NAME, credentials=credentials, col_names=True, row_names=False)

# df_gspread

In [13]:
# Inventory report
# from df2gspread import df2gspread as df2g
data_inventory = data.loc[(data['LoaiBacklog'] == 'Kho giao') & (data['TrangThai'] != 'Đang giao hàng')]
data_inventory['LoaiXuLy'] = 'Chưa giao lại'
chuagiao_filter = data_inventory['ThoiGianGiaoLanDau'].isnull()
giaoloi_filter = ~data_inventory['GhiChuGHN'].isnull() & data_inventory['GhiChuGHN'].str.contains(datetime.today().strftime('%d/%m/%Y'))
data_inventory.loc[chuagiao_filter, 'LoaiXuLy'] = 'Chưa giao lần nào'
data_inventory.loc[giaoloi_filter, 'LoaiXuLy'] = 'Giao lỗi'

def convert_to_day(x):
    try:
        return x.strftime('%Y-%m-%d')
    except ValueError:
        return np.nan

data_inventory['N_ve_kho'] = data_inventory['N0'].apply(lambda x: convert_to_day(x))

del data_inventory['N0']
del data_inventory['N+']
del data_inventory['Aging']
del data_inventory['NgayHienTai']
del data_inventory['GhiChu']
# data_inventory
# to csv file
# data_inventory.to_csv('rp_giao.csv', index=False, encoding='utf-8-sig')
# data_inventory.to_excel('report_giao.xlsx')

# pivot data and push to google sheet
table1 = pd.pivot_table(data_inventory, values='MaDH', index=['LoaiXuLy'], aggfunc='count')
table2 = pd.pivot_table(data_inventory, values='MaDH', index=['LoaiXuLy'], columns=['N_ve_kho'], aggfunc='count', fill_value=0)

# table1
WKS_INVENTORY = 'RPGiao_bot'
# inventory_wks = gc.open_by_key(SPREADSHEET_ID).worksheet(WKS_INVENTORY)
# # inventory_wks.clear()  # clear old data
df2g.upload(
    table1,
    SPREADSHEET_ID,
    WKS_INVENTORY,
    credentials=credentials,
    row_names=True,
    col_names=True,
    start_cell='A1',
)

# inventory_wks.add_row(table2.shape[0])

df2g.upload(
    table2,
    SPREADSHEET_ID,
    WKS_INVENTORY,
    credentials=credentials,
    row_names=True,
    col_names=True,
    clean=False,
    start_cell='A10',
)
# then write detail inventory data
inventory_upload = data_inventory[[
    'MaDH',
    'KhoHienTai',
    'TrangThai',
    'GhiChuGHN',
    'SoLanGiao',
    'ECommerces',
    'LoaiXuLy',
    'N_ve_kho',
    'TrangThaiLuanChuyen',
    'MaKien',
]]
# export to csv
inventory_upload.to_csv('rp_giao.csv', encoding='utf-8-sig', index=False)

df2g.upload(
    inventory_upload,
    SPREADSHEET_ID,
    WKS_INVENTORY,
    credentials=credentials,
    row_names=False,
    col_names=True,
    clean=False,
    start_cell='A15',
)

<Worksheet 'RPGiao_bot' id:1115912348>

In [14]:
# from datetime import datetime

# dt = datetime.strptime('08/08/2020 14:10:12', '%d/%m/%Y %H:%M:%S')
# print(dt)
# def iter_df(df):
#     """Generator for df."""
#     for val in df.columns:
#         yield val
#     for row in df.to_numpy():
#         for val in row:
#             if pd.isna(val):
#                 yield ''
#             else:
#                 yield val